In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [21]:
# --- 1. Cargar datos
df = pd.read_csv("ventas_antimicrobial.csv")
df['fecha'] = pd.to_datetime(df['fecha'])

# --- 2. Asignar industria manualmente
industria_map = {
    'cmpc': 'forestal_construccion',
    'lippi': 'textil',
    'petco': 'mascotas'
}
df['industria'] = df['cliente'].map(industria_map)

# --- 3. Features de fecha
df['mes'] = df['fecha'].dt.month
df['año'] = df['fecha'].dt.year
df['trimestre'] = df['fecha'].dt.quarter
df['es_fin_de_mes'] = df['fecha'].dt.is_month_end.astype(int)

# --- 4. Features del cliente (ventas anuales acumuladas por cliente)
ventas_cliente = (
    df.groupby(['cliente', 'año'])['unidades']
    .sum().reset_index().rename(columns={'unidades': 'ventas_anuales_cliente'})
)
df = df.merge(ventas_cliente, on=['cliente', 'año'], how='left')

# --- 5. Lag feature (ventas del mes anterior)
df = df.sort_values(by=['cliente', 'fecha'])
df['lag_1'] = df.groupby('cliente')['unidades'].shift(1)
df = df.dropna()  # eliminar filas sin lag

# --- 6. Encoding
le_cliente = LabelEncoder()
le_industria = LabelEncoder()

df['cliente_enc'] = le_cliente.fit_transform(df['cliente'])
df['industria_enc'] = le_industria.fit_transform(df['industria'])


# --- 7. Features finales
features = [
    'cliente_enc', 'industria_enc', 'mes', 'año', 'trimestre',
    'es_fin_de_mes', 'ventas_anuales_cliente', 'lag_1'
]
X = df[features]
y = df['unidades']


# --- 8. Split y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4)
model.fit(X_train, y_train)

# --- 9. Predicción
y_pred = model.predict(X_test)

# --- 10. Evaluación global
mae = mean_absolute_error(y_test, y_pred)
print(f"🔍 MAE global: {mae:.2f}")


🔍 MAE global: 12.08


In [22]:
import plotly.graph_objects as go
import plotly.colors as pc

X_test_plot = X_test.copy()
X_test_plot['fecha'] = df.loc[X_test.index, 'fecha']
X_test_plot['cliente'] = df.loc[X_test.index, 'cliente']
X_test_plot['industria'] = df.loc[X_test.index, 'industria']
X_test_plot['y_true'] = y_test
X_test_plot['y_pred'] = y_pred
# Agrupar por cliente para graficar cada uno por separado

clientes = X_test_plot['cliente'].unique()
color_map = dict(zip(clientes, pc.qualitative.Set2))
fig = go.Figure()

for cliente in clientes:
    cliente_df = X_test_plot[X_test_plot['cliente'] == cliente].sort_values(by='fecha')
    color = color_map[cliente]

    # Real (línea sólida)
    fig.add_trace(go.Scatter(
        x=cliente_df['fecha'],
        y=cliente_df['y_true'],
        mode='lines+markers',
        name=f'{cliente} - Real',
        line=dict(dash='solid', color=color),
        marker=dict(symbol='circle')
    ))

    # Predicho (línea punteada, mismo color)
    fig.add_trace(go.Scatter(
        x=cliente_df['fecha'],
        y=cliente_df['y_pred'],
        mode='lines+markers',
        name=f'{cliente} - Predicho',
        line=dict(dash='dot', color=color),
        marker=dict(symbol='x')
    ))

fig.update_layout(
    title='📈 Predicción vs Real por Cliente',
    xaxis_title='Fecha',
    yaxis_title='Unidades',
    legend_title='Serie',
    template='plotly_white'
)

fig.show()


In [23]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# --- 1. Generar fechas objetivo
future_months = pd.date_range(start='2025-01-01', periods=3, freq='MS')

# --- 2. Crear dataset de predicción
clientes_unicos = df['cliente'].unique()
industria_map = df.set_index('cliente')['industria'].to_dict()
cliente_enc_map = dict(zip(le_cliente.classes_, le_cliente.transform(le_cliente.classes_)))
industria_enc_map = dict(zip(le_industria.classes_, le_industria.transform(le_industria.classes_)))

future_rows = []

for cliente in clientes_unicos:
    industria = industria_map[cliente]
    cliente_enc = cliente_enc_map[cliente]
    industria_enc = industria_enc_map[industria]

    # Última venta registrada del cliente
    historico_cliente = df[df['cliente'] == cliente].sort_values('fecha')
    ultima_fecha = historico_cliente['fecha'].max()

    # Obtener lag_1 desde diciembre 2024
    lag_1 = historico_cliente[historico_cliente['fecha'] == ultima_fecha]['unidades'].values[0]

    # Ventas anuales 2024
    ventas_2024 = historico_cliente[historico_cliente['año'] == 2024]['unidades'].sum()

    for fecha in future_months:
        future_rows.append({
            'cliente': cliente,
            'cliente_enc': cliente_enc,
            'industria_enc': industria_enc,
            'fecha': fecha,
            'mes': fecha.month,
            'año': fecha.year,
            'trimestre': ((fecha.month - 1) // 3) + 1,
            'es_fin_de_mes': int(pd.Timestamp(fecha).is_month_end),
            'ventas_anuales_cliente': ventas_2024,
            'lag_1': lag_1
        })

# --- 3. Crear DataFrame futuro
df_future = pd.DataFrame(future_rows)

# --- 4. Predicción
X_future = df_future[features]
df_future['pred_unidades'] = model.predict(X_future)

# --- 5. Visualización
# import plotly.express as px

# fig = px.line(
#     df_future,
#     x='fecha',
#     y='pred_unidades',
#     color='cliente',
#     markers=True,
#     title='🔮 Predicción de ventas Q1 2025 por cliente',
#     labels={'pred_unidades': 'Unidades Predichas'}
# )
# fig.show()


In [24]:
import plotly.express as px

# Convertir mes a nombre para visualización clara
df_future['mes_nombre'] = df_future['fecha'].dt.strftime('%b')

# Orden correcto de meses
orden_meses = ['Jan', 'Feb', 'Mar']

fig = px.bar(
    df_future,
    x='mes_nombre',
    y='pred_unidades',
    color='cliente',
    barmode='group',
    category_orders={'mes_nombre': orden_meses},
    labels={'mes_nombre': 'Mes', 'pred_unidades': 'Unidades Predichas'},
    title='🔮 Predicción de ventas por cliente - Q1 2025 (agrupado por mes)'
)

fig.update_layout(
    xaxis_title='Mes',
    yaxis_title='Unidades Predichas',
    legend_title='Cliente',
    template='plotly_white'
)

fig.show()


In [25]:
# Asegurar que el nombre del mes sea categórico y ordenado
df_future['mes_nombre'] = pd.Categorical(
    df_future['fecha'].dt.strftime('%b'),
    categories=['Jan', 'Feb', 'Mar'],
    ordered=True
)

# Crear gráfico de barras apiladas (stacked)
fig = px.bar(
    df_future,
    x='mes_nombre',
    y='pred_unidades',
    color='cliente',
    barmode='stack',
    labels={'mes_nombre': 'Mes', 'pred_unidades': 'Unidades Predichas'},
    title='📦 Predicción de demanda total por mes (stacked por cliente)'
)

fig.update_layout(
    xaxis_title='Mes',
    yaxis_title='Unidades Predichas',
    legend_title='Cliente',
    template='plotly_white'
)

fig.show()


In [29]:
df_future[['cliente','pred_unidades']].to_csv("prediction_sales.csv", index=False)